# Suivis Projet Info : Stellar Analytics
#### Axelle Refeyton . Jean Roelens

Le but de se projet est initialement de permettre à un utilisateur de prendre une photo d'une étoile et de retrouver des informations dessus, notamment sont nom, sa position ainsi que sa composition chimique qui peux être récupérer en fonction de son spectre lumineux.

Ressources utilisées : 
- [NIST Spectra Database](https://physics.nist.gov/PhysRefData/ASD/lines_form.html)
- [Documentation sur les spectres lumineux](https://atomic-spectra.net/)
- [Simbad, DB d'étoiles communautaire open source](https://simbad.u-strasbg.fr/simbad/)
- [MAST portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html)
- [Raies de Fraunhofer](https://fr.wikipedia.org/wiki/Raies_de_Fraunhofer)

En début de projet beaucoup de temps fut consacré à de la recherche de documentation ainsi que de jeux de données. Egalement nous avons eu l'occasion de nous entretenir lors d'une conférence Zoom avec deux membres du CNES (Orphée Faucoz et Denis Standarovski) travaillant depuis plusieurs années sur un sujet similaire : L'identification d'exoplanétes en utilisant les spectres d'absorption.

Au cours de l'entretien ils nous ont clairement expliqué que ce que l'on souhaiter faire êtait une tâche difficile et que cela nécessité du temps ne serait-ce que pour la compréhension de l'environnement dans lequel nous naviguons.

On a parcouru beaucoup de base de données gratuite et d'autres non afin de trouver celles qui correspondrait au mieux à nos besoins. Au final on a retenue Simbad pour la récupération des spectres émis par les étoiles et NIST pour récupérer le spectre d'émissions des élèments que peuvent composer une étoile, càd : Oxygéne, Hydrogénes, Sodium, Hélium, Mercure, Fer, Magnésium, Calcium, Titane et finalement Nickel.

à partir de ce moment on a divisé la structure du projet en deux parties, une partie objet céleste (gérer par Axelle Refeyton) et une partie spectre chimique (gérer par Jean Roelens). Ensuite chaque partie est divisés en étapes afin de faciliter le procéder de réalisation.

## Spectre Chimique (J.Roelens)

L'objectif de cette partie est :
1. **Pour chaque élèments récupérer les différentes ionization possible.**
2. **Extraire le tableau de spectre depuis NIST (en utilisant un scrapper)**
3. **Le formatter pour récupérer que les lignes fortes**
4. **Créer un dataset par élément et ionization avec les lignes fortes**

Pour commencer il faut que je récupére les ionizations de chaque élèments, étant une étape pas forcément très complexe j'ai préféré le faire à la main en m'aidant de [atomic-spectra](https://atomic-spectra.net/spectrum.php) et des [Raies de Fraunhofer](https://fr.wikipedia.org/wiki/Raies_de_Fraunhofer) On en sort cette liste : 
- Oxygéne [I, II, III, IV, V, VI, VII, VIII]
- Hydrogénes [I]
- Sodium [I, II, III, IV, V, VI, VII, VIII, IX, X, XI]
- Hélium [I, II]
- Mercure [I, II, III]
- Fer [I, II ,III , IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI, XVII, XVIII, XIX, XX, XXI, XXII, XXIII, XXIV, XXV, XVI]
- Magnésium [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII]
- Calcium [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI]
- Titane [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI, XVII, XVIII, XIX, XX, XXI, XXII]
- Nickel [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI, XVII, XVIII, XIX, XX, XXI, XXII, XXIII, XXIV, XXV, XXVI, XXVII, XXVIII]

Maintenant que l'étape facile est faites je vais pouvoir m'attaquer à l'étape plus hardu d'extraire les données depuis NIST. L'idée global dérriére est de récupérer le HTML de la page NIST en donnant dans l'URL du site l'élèment ainsi que son indice de ionization.

En saisissant H I (pour Hydrogénes Ionization 1) on obtiens cet URL :

https://physics.nist.gov/cgi-bin/ASD/lines1.pl?spectra=H+I&output_type=0&low_w=&upp_w=&unit=1&submit=Retrieve+Data&de=0&plot_out=0&I_scale_type=1&format=0&line_out=0&en_unit=0&output=0&bibrefs=1&page_size=15&show_obs_wl=1&show_calc_wl=1&unc_out=1&order_out=0&max_low_enrg=&show_av=2&max_upp_enrg=&tsb_value=0&min_str=&A_out=0&intens_out=on&max_str=&allowed_out=1&forbid_out=1&min_accur=&min_intens=&conf_out=on&term_out=on&enrg_out=on&J_out=on

Plutôt long, mais ce qui est assez intéressant que beaucoup de paramètres pourront-être passé assez facilement par celles-ci comparons avec l'URL de O I pour voir ou nous devrons saisir l'élèment et l'ionization 

https://physics.nist.gov/cgi-bin/ASD/lines1.pl?spectra=O+I&output_type=0&low_w=&upp_w=&unit=1&submit=Retrieve+Data&de=0&plot_out=0&I_scale_type=1&format=0&line_out=0&en_unit=0&output=0&bibrefs=1&page_size=15&show_obs_wl=1&show_calc_wl=1&unc_out=1&order_out=0&max_low_enrg=&show_av=2&max_upp_enrg=&tsb_value=0&min_str=&A_out=0&intens_out=on&max_str=&allowed_out=1&forbid_out=1&min_accur=&min_intens=&conf_out=on&term_out=on&enrg_out=on&J_out=on

Comme on peut le voir l'élèment se trouve en début d'URL essayons cela.

In [1]:
import requests

START_URL = r"https://physics.nist.gov/cgi-bin/ASD/lines1.pl?spectra="
END_URL = r"&output_type=0&low_w=&upp_w=&unit=1&submit=Retrieve+Data&de=0&plot_out=0&I_scale_type=1&format=0&line_out=0&en_unit=0&output=0&bibrefs=1&page_size=15&show_obs_wl=1&show_calc_wl=1&unc_out=1&order_out=0&max_low_enrg=&show_av=2&max_upp_enrg=&tsb_value=0&min_str=&A_out=0&intens_out=on&max_str=&allowed_out=1&forbid_out=1&min_accur=&min_intens=&conf_out=on&term_out=on&enrg_out=on&J_out=on"

# Essayons de voir si on peut obtenir un code 200, mais avant cela construisons notre URL de maniére naïve pour ce test

URL = START_URL + "Ni+X" + END_URL

resp = requests.get(URL)
if resp.status_code == 200:
    print(f"Succés de la requéte")
else :
    print(f"Echec de la requéte")

Succés de la requéte


Maintenant que notre approche naïve fonctionne vérifions que l'intégralité des élèments et ionization fonctionne.

In [2]:
# Le fait que je marque certaines "variables" en majuscules est simplement un tic pour m'indiquer qu'il s'agit de constante
ELEMENT_DICT = {
    "O" : ["I", "II", "III", "IV", "V", "VI","VII"],
    "H" : ["I"],
    "Na" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI"],
    "He" : ["I","II"],
    "Hg" : ["I","II","III"],
    "Fe" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI"],
    "Mg" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII"],
    "Ca" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI"],
    "Ti" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI","XVII","XVIII","XIX","XX","XXI","XXII"],
    "Ni" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI","XVII","XVIII","XIX","XX","XXI","XXII","XXIII","XXIV","XXV","XXVI","XXVII","XXVIII"]
}

# Je vérifie la longueur de chacune des clès pour être sur de ne pas avoir fait d'erreur 
def element_dict_verification():
    for key in ELEMENT_DICT.keys():
        print(f"Elément : {key} Len : {len(ELEMENT_DICT[key])}")

# Et ensuite on test les requêtes

def get_200(r : requests.models.Response):
    if r.status_code != 200:
        return False
    return True
def multi_request_verification(d : dict):
    for k in d.keys():
        for v in range(len(d[k])):
            url = START_URL + k + "+" + d[k][v] + END_URL
            if not get_200(requests.get(url)):
                print(f"Error with el : {k} and ionization : {d[k][v]}")
            else : 
                print(f"Success with el : {k} and ionization : {d[k][v]}")
# multi_request_verification(ELEMENT_DICT)

Ok on dirait bien qu'on a que des codes 200 pour l'ensemble des requêtes ! Malheuresement on se rendra compte plus tard que NIST renvoie des code 200 même en cas de mauvais URL. Le seul moyen de savoir si la requête a bien aboutit et de récupérer le tableau **"\<table>"** qui va contenir les données qui nous intéresse. 

L'idée va être de voir si on peut récupérer la balise "table" contenant le styling que nous cherchons pour chaque lien avec l'aide de la librairie que nous allons le plus utiliser durant notre partie : "BeautifulSoup"

In [3]:
from bs4 import BeautifulSoup
from pprint import pprint

# Vu que je vais beaucoup utiliser request autant se créer des petites fonctions QOL (Quality Of Life)

def url_builder(d : dict):
    resp = {}
    for key in d.keys():
        for v in d[key]:
            resp[key+v] = START_URL + key + "+" + v + END_URL
    return resp

In [6]:
urls = url_builder(ELEMENT_DICT)
for url in urls:
    req = requests.get(urls[url])
    soup = BeautifulSoup(req.text, 'html.parser')
    tables = soup.find_all('table')
    target_table = [table for table in tables if 'background-color:#FFFEEE;' in table.get('style', '')]
    if len(target_table) != 1:
        print(f'error with url : {url}')
    else :
        continue

error with url : NiVI
error with url : NiVIII


On voit donc désormais que deux lien sont *mort* celui pour le Nickel 6 et le Nickel 8. Hormis ça l'ensemble des autres liens marche. Pour la prochaine étape il nous faut récupérer toute les rows contenant une donnée dans la colonne *Observed Wavelength Vac (nm)* ainsi que *Rel. Int.* C'est les deux colonnes qui nous intéresse pour la suite.

In [ ]:
# Pour commencer passont la logique précédente en fonction 

def get_table(req : requests.models.Response):
    soup = BeautifulSoup(req.text,'html.parser')
    tables = soup.find_all('table')
    target_table = [table for table in tables if 'background-color:#FFFEEE;' in table.get('style', '')]
    if len(target_table) > 0 :
        return target_table[0]
    else :
        return None

# On va traiter seulement le premier question de rapidité depuis python 3.6 les dicts sont ordonnées du coup le premier élèment sera toujours OI
urls = url_builder(ELEMENT_DICT)
oxygen_one = list(urls.items())[0]
table = get_table(requests.get(oxygen_one[1]))
# Maintenant on va "drill down" la table afin d'obtenir tout les Tbody puis les Tr qui nous intéressent.
tbodies = table.find_all('tbody')
# Seul les tbody pair nous intéresse car ils contiennent la data en question
evenTBodies = tbodies[1::2]
# Ensuite pour chaque tbody on veut récupérer toute les balises tr
# firstTBody = evenTBodies[0]
# Holder pour les values récupéré afin d'en faire un dataframe par la suite
All_Observed_Wavelength = []
All_Relative_Intensities = []

# for tbody in evenTBodies :
#     for tr in tbody.find_all('tr'):
#         all_td = tr.find_all('td')
#         if len(all_td) >= 2:
#             Obs_Wav = all_td[0].text.strip()
#             Rel_Int = all_td[2].text.strip()
#             if Obs_Wav != "" and Rel_Int != "":
#                 All_Observed_Wavelength.append(Obs_Wav)
#                 All_Relative_Intensities.append(Rel_Int)
# pprint(All_Observed_Wavelength)

# Tiens on remarque que la structure du HTML doit avoir un petite erreur car lors du print de All_Observed_Wavelength 
# on remarque qu'on print les bonnes colonnes avec les bonnes valeur mais, on a 1 fois le tableau 1, 2 fois le tableau 2, et finalement 3 fois le tableau
# 3 ce qui nous indique que l'on peut parcourir une seul fois le tableau 1 et récupérer toutes les données.

tbody = tbodies[1]
for tr in tbody.find_all('tr'):
    all_td = tr.find_all('td')
    if len(all_td) >= 2:
        obs_wav = all_td[0].text.strip()
        rel_int = all_td[2].text.strip()
        if obs_wav != "" and rel_int != "":
            All_Observed_Wavelength.append(obs_wav)
            All_Relative_Intensities.append(rel_int)
import pandas as pd

df = pd.DataFrame({
    'wavelength' : All_Observed_Wavelength,
    'intensities': All_Relative_Intensities,
    'chemical' : 'O I'
})

print(df.describe())

       wavelength intensities chemical
count         174         174      174
unique        157          48        1
top       94.8686         120      O I
freq            3          18      174


## Création de spectre (A.Refeyton)

<font color='#FF5733'>Attention cette partie comportera des blocs de code mais ceux-ci seront plus à titre indicatif que réellement fonctionnel étant donné le besoin des jeux de donnèes pour que ceux-ci fonctionne.</font>

Pour la création de spectre l'idée va être de re exploiter les donnèes scrapper de maniére à sélectionner parmie elles un nombre X d'éléments et de créée une sorte de copycat de spectres d'étoiles à partir de ces élèments.

Pour commencer il nous faut récupérer la liste des éléments, ceux ci sont stocké directement dans le nom des csv.

In [15]:
import os
import pandas as pd

def get_elements_list(path):
    """
    path : chemin vers le dossier contenant tout les csv éléments
    """
    filenames = os.listdir(path)
    dataframes = []
    for n in filenames:
        data = pd.read_csv(path + "\\" + n)
        # On nomme le dataframe en fonction de l'élément
        data.Name = n[:-4]
        dataframes.append(data)
    return dataframes

Une fois les dataframes récupérer et nommé on va pouvoir donc les exploiter et générer nos spectres.

In [16]:
import numpy as np
import random as rand

def generate_spectrums(base_elements, n):
    """
    base_elements : Ensemble des dataframes éléments
    n : nombre de dataframe à générer
    """
    max_elems = len(base_elements)
    list_df = []
    matrice_labels = pd.DataFrame(columns=[x.Name for x in base_elements]) 

    # On vérifie que plus de 2 éléments soit présent
    if max_elems < 2:
        raise ValueError(f'Not enough data, {max_elems} elements present')
    for i in range(n):
        # création du df resultat en y ajoutant un nombre aléatoire d'éléments de base compris entre 2 et 5
        rand_elems = rand.choices(base_elements, k=rand.randrange(2, max_elems if max_elems < 5 else 5))
        # initialisation des columns
        matrice_labels.loc[i] = [0] * max_elems

        # Changement du 0 en 1 pour les éléments présents
        for elem in rand_elems:
            matrice_labels.at[i, elem.Name] = 1
            
        df_res = pd.concat(rand_elems, ignore_index=True)

        # Génération du bruit
        n_noise = rand.randint(1500, 2000)
        # Génération des wavelengths pour chacun des bruits
        rand_wls = [rand.uniform(0,1100) for x in range(n_noise)]

        # Ajout des noise au dataset existant
        for n in range(0,n_noise):
            df_res.loc[len(df_res)] = {"wavelength": round(rand_wls[n], 1), "relint": round(rand.uniform(0, 50), 1)} 
        
        list_df.append(df_res.sort_values("wavelength"))
        
    return list_df, matrice_labels

Nous avons donc maintenant des spectres fabriqués par nos soins avec du bruit de maniére à rendre le résultat plus proche du réel.
Il est temps de l'exploiter.

## Exploitation des spectres construis

Attaquons nous à la partie qui nous intéresse, exploiter les spectres de maniére à voir si on est capable de trouver les différents éléments d'un spectre avec un modèle.

Pour ça nous allons devoir utiliser un type de modèle un peu particulier, celui de multi-labelling classification. Il s'agit de modèle capable de classifier une observation selon plusieurs labels.

La premiére approche va être avec un modéle unique puis nous verrons avec plusieurs modéle en même temps (pour un gain de temps).

Mais avant d'exploiter les modèles il nous faut encore quelques fonctions qui vont nous servir pour tout ce qui va être reshaping des données.

In [8]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import warnings # Permet d'ignorer certains warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

Pour pouvoir utiliser le PCA (Principal Component Analysis) qui nous permettra d'isoler les informations de spectres qui nous intéresse, et gagner en temps de process. Il faut d'abord que notre spectre soit interprété comme un array de dimension 1.

In [9]:
def shape(list_df):
    result_df = pd.DataFrame()

    for df in list_df:
        # Aplatir le df en liste de x valeurs
        data_flat = df.to_numpy().flatten()

        # ajouter des colonnes au dataframe s'il n'y en a pas assez
        while len(data_flat) > len(result_df.columns):
            result_df[len(result_df.columns)] = 0

        # Si le df aplatis à moins de colonnes que le df de résultat on rajoute des colonnes au df aplatis et on y attribue la valeur nan (0)
        if len(data_flat) < len(result_df.columns):
            data_flat = np.concatenate([data_flat, [0] * (len(result_df.columns) - len(data_flat))])
        
        # ajout de la ligne
        result_df.loc[len(result_df)] = data_flat

    return result_df

Maintenant nous allons pouvoir mettre en place notre PCA.

In [10]:
def pca(df, nb_components):

    # Transforme le dataframe pour que les valeurs soit comprise en 0 et 1
    data_scaled = MinMaxScaler().fit(df).transform(df)

    # PCA 
    result_df = PCA(n_components=nb_components).fit_transform(data_scaled)

    return result_df

Pas très impressionant en effet et pourtant c'est cette méthode qui va nous faire gagner un bon nombre de temps, il nous manque une petite fonction qui sera plus la en tant que fonction QOL (quality of life) et qui rendra juste le résultat final plus lisible.

In [11]:
def encode_labels(labels):
    le = LabelEncoder()
    data_encoded = le.fit_transform(labels)
    return data_encoded, le

Bien maintenant que tout est setup nous allons pouvoir mettre en place la logique pour le modèle.

<font color='#FF5733'>Les modèles ne tourneront pas sur le Jupyter Notebook pour des raisons de baisse de performance</font>

In [20]:
from sklearn.model_selection import train_test_split
import xgboost as xgb # Le modéle de multi labelling
from sklearn.metrics import recall_score
from statistics import mean

# Pour commencer on récupére les élements
elements = get_elements_list("./generation_spectre/elements")

# Génére les spectres aléatoires + les labels associés
spectrums, labels = generate_spectrums(elements, 10)

# Transforme la liste de df en 1 df
data_shaped = shape(spectrums)

# On resize la données avec pca
data_scaled = pca(data_shaped, 5)

# On split la données pour finalement essayer notre modèle
X_train, X_val, y_train, y_val = train_test_split(data_scaled, labels.to_numpy(), test_size=0.8, random_state = 42)

model = xgb.XGBClassifier(tree_method="hist")
model.fit(X_train, y_train)
prediction = model.predict(X_val)

# Calculation de précision
overall_precision = ['%.2f' % elem for elem in recall_score(y_val, prediction, average=None)]
positive_precision = []
for i in range(len(y_val)):
    for j in range(len(y_val[i])):
        if y_val[i][j] == 1:
            positive_precision.append(prediction[i][j])
positive_precision = mean(positive_precision)

print(f"Score by label : {overall_precision}")
print(f"Positive labels precision : {positive_precision}")


Score by label : ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
Positive labels precision : 0.0


C:\Users\littl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Sans surprise avec peu de données généré le score est nulle (pour le coup vraiment nulle car on est à 0) mais sur des jeux plus gros et avec plus de temps on a réussi à obtenir 11% avec 2000 spectres et 13,8% avec 7000 spectres.
Mais on a utiliser une deuxiéme méthode qui utilise plusieurs modèles, chaque modèle va travailler sur un lot de label plutôt que tous en même temps.

Mettons ça en place.

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from statistics import mean
from sklearn.metrics import recall_score

max_labels_per_df = 5

# Les premières étapes sont similaires
# On récupére les éléments
elements = get_elements_list("./generation_spectre/elements")

# Génére les spectres aléatoires + les labels associés
spectrums, labels = generate_spectrums(elements, 10)

# Transforme la liste de df en 1 df
data_shaped = shape(spectrums)

# On resize la données avec pca
data_scaled = pca(data_shaped, 5)

# On slice ensuite les labels pour que chaque modèle s'occupe d'un morceau
list_sliced_labels = []

for i in range(len(labels.columns)//max_labels_per_df):
    list_sliced_labels.append(labels.iloc[:,i * max_labels_per_df : (i+1) * max_labels_per_df])

# Add remaining columns in a last df 
list_sliced_labels.append(labels.iloc[:, -(len(labels.columns) % max_labels_per_df) :])

# Model intefration : 
prevision = pd.DataFrame()
expected = pd.DataFrame()

for df in list_sliced_labels:
    # Préparation des données
    X_train, X_val, y_train, y_val = train_test_split(data_scaled, df.to_numpy(), test_size=0.8, random_state=42)

    # Training
    model = XGBClassifier(tree_method="hist")
    model.fit(X_train, y_train)

    # Predicting
    result = model.predict(X_val)
    prevision = pd.concat([prevision, pd.DataFrame(result)], axis=1)
    expected = pd.concat([expected, pd.DataFrame(y_val)], axis=1)

# Finalement on calcul la précision comme avant
prevision = prevision.to_numpy()
expected = expected.to_numpy()
map(float, overall_precision)
positive_precision = []
for i in range(len(expected)):
    for j in range(len(expected[i])):
        if expected[i][j] == 1:
            positive_precision.append(prediction[i][j])
positive_precision = mean(positive_precision)

print(f"positive labels precision : {positive_precision}")

positive labels precision : 0.0


Encore une fois un résultat nulle mais avec un jeux de données de 2000 on obtiens un score de 12% et 14% avec 7000 spectres.